In [1]:
import os
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import plotly.graph_objects as go
from torch.utils.data import DataLoader

def minmax_scale(tensor):
    tensor_min = tensor.min()
    tensor_max = tensor.max()
    scaled_tensor = (tensor - tensor_min) / (tensor_max - tensor_min)
    return scaled_tensor, tensor_min, tensor_max

def inverse_minmax_scale(scaled_tensor, tensor_min, tensor_max):
    original_tensor = scaled_tensor * (tensor_max - tensor_min) + tensor_min
    return original_tensor
    
def smape(y_true, y_pred):
    return 100 * torch.mean(2 * torch.abs(y_true - y_pred) / (torch.abs(y_true) + torch.abs(y_pred))).item()

def rmse(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2))
    
def mae(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))


window_size = 24
horizon = 24
train_prop = 0.6
val_prop = 0.2

df1 = pd.read_csv('detached1.csv')
df2 = pd.read_csv('detached2.csv')
df1 = pd.read_csv('detached1.csv').drop(columns=['Datetime'])
df2 = pd.read_csv('detached2.csv').drop(columns=['Datetime'])

class SlidingWindowsDataset(torch.utils.data.Dataset):
    def __init__(self, data, window_size=24, horizon=24):
        self.data = data
        self.window_size = window_size
        self.horizon = horizon
        
        self.feature_mins = self.data.min(axis=0).values
        self.feature_maxs = self.data.max(axis=0).values
    
    def __len__(self):
        return len(self.data) - (self.window_size + self.horizon) + 1

    def __getitem__(self, idx):
        window = self.data[idx: idx + self.window_size + self.horizon]
        scaled_past_all_features = window[:self.window_size]
        future_energy = window[self.window_size:, -1:]
        
        #scaled_past_all_features = (past_all_features - self.feature_mins) / (self.feature_maxs - self.feature_mins)
        
        scaled_future_energy, energy_min, energy_max = minmax_scale(future_energy)

        return scaled_past_all_features, scaled_future_energy, (energy_min, energy_max)

import torch

df_dict = {
    'Detached_House_1': df1,
    'Detached_House_2':df2
}

train_datasets = {}
val_datasets = {}
test_datasets = {}

for dataset_name, dataset_df in df_dict.items():
    # Convert to tensor and perform the split
    data_tensor = torch.from_numpy(dataset_df.values).float()

    data_shape = data_tensor.shape[0]
    train_end = int(data_shape * train_prop)
    val_end = int(data_shape * (train_prop + val_prop))

    train_data = data_tensor[:train_end]
    val_data = data_tensor[train_end:val_end]
    test_data = data_tensor[val_end:]
    
    train_datasets[dataset_name] = SlidingWindowsDataset(train_data, window_size, horizon)
    val_datasets[dataset_name] = SlidingWindowsDataset(val_data, window_size, horizon)
    test_datasets[dataset_name] = SlidingWindowsDataset(test_data, window_size, horizon)

for dwelling_type, dataset in train_datasets.items():
    item = dataset[0]  # This gets the first item in the dataset
    past_features, future_energy, (energy_min, energy_max) = item
    
    print(f"Data for {dwelling_type}:")
    print("Past Features:")
    print(past_features)
    print("Future Energy:")
    print(future_energy)
    print("Energy Min:", energy_min.item())
    print("Energy Max:", energy_max.item())
    print("\n" + "="*50 + "\n")


datasets = {
    'train': train_datasets,
    'val': val_datasets,
    'test': test_datasets
}


Data for Detached_House_1:
Past Features:
tensor([[2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.8000e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.9600e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 2.0000e+00, 1.8000e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 3.0000e+00, 1.9000e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 4.0000e+00, 1.7900e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 5.0000e+00, 1.8300e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 6.0000e+00, 1.8400e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 7.0000e+00, 1.9800e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 8.0000e+00, 2.1100e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 9.0000e+00, 2.1400e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 1.0000e+01, 2.0200e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00,

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, dropout=0.8)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)  
        return out

class GRU(nn.Module):
    def __init__(self, num_features):
        super(GRU, self).__init__()       
        self.gru = nn.GRU(input_size=num_features, hidden_size=64, batch_first=True,dropout=0.1)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x, _ = self.gru(x)
        output = self.fc(x)
        
        return output


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()  
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(1, 0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Transformer(nn.Module):
    def __init__(self, num_features, d_model, nhead, num_layers):
        super(Transformer, self).__init__()
        self.embedding = nn.Linear(num_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer.encoder(x)
        output = self.fc(x)
        return output



device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

if device.type == 'cuda':
    print("Training will be performed on GPU")
else:
    print("Training will be performed on CPU")


lstm = LSTM(input_size=6, hidden_size=32, output_size=1).to(device)
gru = GRU(num_features=6).to(device)
transformer = Transformer(num_features=6, d_model=128, nhead=4, num_layers=2).to(device)

Training will be performed on GPU


/usr/lib/python3/dist-packages/torch/nn/modules/rnn.py:71: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1

/usr/lib/python3/dist-packages/torch/nn/modules/rnn.py:71: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1



In [6]:
import os

import plotly.graph_objects as go

def combined_test(model, data_loader, loss_fn):
    model.eval()
    losses = []
    all_outputs = []
    all_targets = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}
    
    first_iter = True  

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scaled_targets)
            losses.append(loss.item())

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max)

            metrics['MAE'].append(mae(outputs, targets).item())
            metrics['RMSE'].append(rmse(outputs, targets).item())
            metrics['SMAPE'].append(smape(outputs, targets))

            all_outputs.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

            if first_iter:  
                print(f"Output shape: {outputs.shape}")
                print(f"Target shape: {targets.shape}")
                first_iter = False  

    test_loss = np.mean(losses)
    test_metrics = {k: np.mean(v) for k, v in metrics.items()}

    return test_loss, test_metrics, all_targets, all_outputs

models_list = {
    "GRU": gru,
    "LSTM": lstm,
    "Transformer": transformer,
}

def combined_testing_and_visualization(datasets, models_list, loss_fn):
    for model_name, model in models_list.items():
        print(f"\nTesting {model_name}...\n")
        for dataset_name in datasets['test'].keys():
            print(f"Testing and generating values for dataset: {dataset_name} using {model_name}")
            
            # Load the trained model's state_dict
            model.load_state_dict(torch.load(f'./Combine_MSE/{model_name}_{loss_fn}_{dataset_name}.pth'))
            model.to(device)

            test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)
            test_loss, test_metrics, actual_values, predicted_values = combined_test(model, test_loader, loss_fn)
            
            print(f"Test Loss for {dataset_name} using {model_name}: {test_loss:.2f}")
            print(f"Test MAE for {dataset_name} using {model_name}: {test_metrics['MAE']:.2f}")
            print(f"Test RMSE for {dataset_name} using {model_name}: {test_metrics['RMSE']:.2f}")
            print(f"Test SMAPE for {dataset_name} using {model_name}: {test_metrics['SMAPE']:.2f}")
            print('-' * 50)

            df_results = pd.DataFrame({
                'Actual': np.ravel(actual_values),
                'Predicted': np.ravel(predicted_values)
            })
            df_results.to_csv(f"./Combine_MSE/{dataset_name}_{model_name}_results.csv", index=False)

            df_results = pd.read_csv(f"./Combine_MSE/{dataset_name}_{model_name}_results.csv")

            actual_values = df_results['Actual'].values[:600]
            predicted_values = df_results['Predicted'].values[:600]

            fig = go.Figure()
            fig.add_trace(go.Scatter(y=actual_values, mode='lines', name='Actual Values'))
            fig.add_trace(go.Scatter(y=predicted_values, mode='lines', name='Predicted Values'))
            
            # Dynamic y-axis adjustment
            max_value = max(np.max(actual_values), np.max(predicted_values))
            yaxis_max = int((max_value // 50 + 1) * 50)
            
            fig.update_layout(
                title=f"Actual vs Predicted Energy Consumption for {dataset_name} using {model_name}",
                xaxis_title="Time Steps", 
                yaxis_title="Energy Consumption",
                yaxis=dict(
                    tickvals=list(range(0, yaxis_max + 1, 125)),
                    range=[0, max_value]
                ),
                height=300,
            )
            fig.show()

    print("All done!")

loss_fn = nn.L1Loss()
combined_testing_and_visualization(datasets, models_list, loss_fn)


Testing GRU...

Testing and generating values for dataset: Detached_House_1 using GRU
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_1 using GRU: 0.31
Test MAE for Detached_House_1 using GRU: 645.24
Test RMSE for Detached_House_1 using GRU: 717.64
Test SMAPE for Detached_House_1 using GRU: 32.60
--------------------------------------------------


Testing and generating values for dataset: Detached_House_2 using GRU
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_2 using GRU: 0.20
Test MAE for Detached_House_2 using GRU: 371.69
Test RMSE for Detached_House_2 using GRU: 537.68
Test SMAPE for Detached_House_2 using GRU: 31.47
--------------------------------------------------



Testing LSTM...

Testing and generating values for dataset: Detached_House_1 using LSTM
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_1 using LSTM: 0.31
Test MAE for Detached_House_1 using LSTM: 660.32
Test RMSE for Detached_House_1 using LSTM: 748.26
Test SMAPE for Detached_House_1 using LSTM: 33.36
--------------------------------------------------


Testing and generating values for dataset: Detached_House_2 using LSTM
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_2 using LSTM: 0.20
Test MAE for Detached_House_2 using LSTM: 377.99
Test RMSE for Detached_House_2 using LSTM: 545.58
Test SMAPE for Detached_House_2 using LSTM: 32.13
--------------------------------------------------



Testing Transformer...

Testing and generating values for dataset: Detached_House_1 using Transformer
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_1 using Transformer: 0.22
Test MAE for Detached_House_1 using Transformer: 466.95
Test RMSE for Detached_House_1 using Transformer: 590.51
Test SMAPE for Detached_House_1 using Transformer: 22.60
--------------------------------------------------


Testing and generating values for dataset: Detached_House_2 using Transformer
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_2 using Transformer: 0.19
Test MAE for Detached_House_2 using Transformer: 345.84
Test RMSE for Detached_House_2 using Transformer: 516.88
Test SMAPE for Detached_House_2 using Transformer: 29.00
--------------------------------------------------


All done!


In [8]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import plotly.graph_objects as go
from torch.nn import L1Loss

all_metrics = {
    'Model': [],
    'Dataset': [],
    'Loss': [],
    'MAE': [],
    'RMSE': [],
    'SMAPE': []
}

def combined_testing_and_visualization(datasets, models_list, loss_fn):
    for model_name, model in models_list.items():
        print(f"\nTesting {model_name}...\n")
        for dataset_name in datasets['test'].keys():
            print(f"Testing and generating values for dataset: {dataset_name} using {model_name}")

            model.load_state_dict(torch.load(f'./Combine_MSE/{model_name}_{loss_fn}_{dataset_name}.pth'))
            model.to(device)

            test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)
            test_loss, test_metrics, actual_values, predicted_values = combined_test(model, test_loader, loss_fn)

            all_metrics['Model'].append(model_name)
            all_metrics['Dataset'].append(dataset_name)
            all_metrics['Loss'].append(test_loss)
            all_metrics['MAE'].append(test_metrics['MAE'])
            all_metrics['RMSE'].append(test_metrics['RMSE'])
            all_metrics['SMAPE'].append(test_metrics['SMAPE'])

            print(f"Test Loss for {dataset_name} using {model_name}: {test_loss:.2f}")
            print(f"Test MAE for {dataset_name} using {model_name}: {test_metrics['MAE']:.2f}")
            print(f"Test RMSE for {dataset_name} using {model_name}: {test_metrics['RMSE']:.2f}")
            print(f"Test SMAPE for {dataset_name} using {model_name}: {test_metrics['SMAPE']:.2f}")
            print('-' * 50)


    # Convert all_metrics dictionary to DataFrame
    df_all_metrics = pd.DataFrame(all_metrics)

    # Calculate the mean of the metrics
    mean_metrics = df_all_metrics.mean(axis=0)

    print("All Metrics DataFrame:")
    print(df_all_metrics)

    print("\nMean of All Metrics:")
    print(mean_metrics)

loss_fn = nn.MSELoss()
combined_testing_and_visualization(datasets, models_list, loss_fn)


Testing GRU...

Testing and generating values for dataset: Detached_House_1 using GRU
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_1 using GRU: 0.31
Test MAE for Detached_House_1 using GRU: 645.24
Test RMSE for Detached_House_1 using GRU: 717.64
Test SMAPE for Detached_House_1 using GRU: 32.60
--------------------------------------------------
Testing and generating values for dataset: Detached_House_2 using GRU
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_2 using GRU: 0.20
Test MAE for Detached_House_2 using GRU: 371.69
Test RMSE for Detached_House_2 using GRU: 537.68
Test SMAPE for Detached_House_2 using GRU: 31.47
--------------------------------------------------

Testing LSTM...

Testing and generating values for dataset: Detached_House_1 using LSTM
Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Loss for Detached_House_1 using LSTM: 0

/tmp/ipykernel_2601438/1531435752.py:50: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [9]:
import os
import plotly.graph_objects as go
import plotly.express as px  
# Combined testing function
def combined_testing_and_visualization(datasets, models_list, loss_fn):
    metrics_data = {
        'MAE': {},
        'RMSE': {},
        'SMAPE': {}
    }

    for model_name, model in models_list.items():
        for dataset_name in datasets['test'].keys():
            print(f"\nTesting {model_name} on dataset {dataset_name}...\n")
            
            model.load_state_dict(torch.load(f'./Combine_MSE/{model_name}_{loss_fn}_{dataset_name}.pth'))
            model.to(device)

            test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)
            test_loss, test_metrics, actual_values, predicted_values = combined_test(model, test_loader, loss_fn)

            print(f"Test Metrics for {dataset_name} using {model_name}:")
            print(f"MAE: {test_metrics['MAE']:.2f}, RMSE: {test_metrics['RMSE']:.2f}, SMAPE: {test_metrics['SMAPE']:.2f}")
            print('-' * 50)

            for metric in ['MAE', 'RMSE', 'SMAPE']:
                if dataset_name not in metrics_data[metric]:
                    metrics_data[metric][dataset_name] = []
                metrics_data[metric][dataset_name].append(test_metrics[metric])

    for metric in ['MAE', 'RMSE', 'SMAPE']:
        for dataset_name in datasets['test'].keys():
            fig = go.Figure()

            sorted_metrics_data = sorted(list(zip(list(models_list.keys()), metrics_data[metric][dataset_name])),
                                         key=lambda x: x[1])  # Sort the data by metric values
            model_names_sorted, metrics_values_sorted = zip(*sorted_metrics_data)  # Unpack sorted data

            fig.add_trace(go.Bar(
                x=model_names_sorted,
                y=metrics_values_sorted,
                marker_color=px.colors.qualitative.Light24  # Use pastel color scale
            ))

            # Update layout
            fig.update_layout(
                title=f'Comparison of {metric} for {dataset_name}',
                xaxis_title='Models',
                yaxis_title=metric,
                xaxis=dict(tickvals=list(range(len(models_list.keys()))), ticktext=model_names_sorted),
                bargap=0.15,
            )

            fig.show()

    print("All done!")

combined_testing_and_visualization(datasets, models_list, loss_fn)



Testing GRU on dataset Detached_House_1...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_1 using GRU:
MAE: 645.24, RMSE: 717.64, SMAPE: 32.60
--------------------------------------------------

Testing GRU on dataset Detached_House_2...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_2 using GRU:
MAE: 371.69, RMSE: 537.68, SMAPE: 31.47
--------------------------------------------------

Testing LSTM on dataset Detached_House_1...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_1 using LSTM:
MAE: 660.32, RMSE: 748.26, SMAPE: 33.36
--------------------------------------------------

Testing LSTM on dataset Detached_House_2...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_2 using LSTM:
MAE: 377.99, RMSE: 545.58, SMAPE: 32.13
---------------------------

All done!


In [10]:
import os
import plotly.graph_objects as go
import plotly.express as px  

def combined_testing_and_visualization(datasets, models_list, loss_fn):
    metrics_data = {
        'MAE': {},
        'RMSE': {},
        'SMAPE': {}
    }

    for model_name, model in models_list.items():
        for dataset_name in datasets['test'].keys():
            print(f"\nTesting {model_name} on dataset {dataset_name}...\n")
            
            # Load the trained model's state_dict
            model.load_state_dict(torch.load(f'./Combine_MSE/{model_name}_{loss_fn}_{dataset_name}.pth'))
            model.to(device)

            test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)
            test_loss, test_metrics, actual_values, predicted_values = combined_test(model, test_loader, loss_fn)

            # Print test metrics
            print(f"Test Metrics for {dataset_name} using {model_name}:")
            print(f"MAE: {test_metrics['MAE']:.2f}, RMSE: {test_metrics['RMSE']:.2f}, SMAPE: {test_metrics['SMAPE']:.2f}")
            print('-' * 50)

            # Store metrics data for bar chart
            for metric in ['MAE', 'RMSE', 'SMAPE']:
                if dataset_name not in metrics_data[metric]:
                    metrics_data[metric][dataset_name] = []
                metrics_data[metric][dataset_name].append(test_metrics[metric])

    # Create bar chart for each metric and dataset
    for metric in ['MAE', 'RMSE', 'SMAPE']:
        for dataset_name in datasets['test'].keys():
            fig = go.Figure()

            sorted_metrics_data = sorted(list(zip(list(models_list.keys()), metrics_data[metric][dataset_name])),
                                         key=lambda x: x[1])  # Sort the data by metric values
            model_names_sorted, metrics_values_sorted = zip(*sorted_metrics_data)  # Unpack sorted data

            colors = ['#00FE35', '#00FC55', '#00FA75', '#FF9090', '#FF6060', '#FF3030']  # Colors for bars
            ##16FF30
            fig.add_trace(go.Bar(
                x=model_names_sorted,
                y=metrics_values_sorted,
                marker_color=colors  # Assigning colors to bars
            ))

            # Update layout
            fig.update_layout(
                title=f'Comparison of {metric} for {dataset_name}',
                xaxis_title='Models',
                yaxis_title=metric,
                xaxis=dict(tickvals=list(range(len(models_list.keys()))), ticktext=model_names_sorted),
                bargap=0.15,
            )

            fig.show()

    print("All done!")

# Running the combined testing and visualization function
combined_testing_and_visualization(datasets, models_list, loss_fn)



Testing GRU on dataset Detached_House_1...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_1 using GRU:
MAE: 645.24, RMSE: 717.64, SMAPE: 32.60
--------------------------------------------------

Testing GRU on dataset Detached_House_2...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_2 using GRU:
MAE: 371.69, RMSE: 537.68, SMAPE: 31.47
--------------------------------------------------

Testing LSTM on dataset Detached_House_1...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_1 using LSTM:
MAE: 660.32, RMSE: 748.26, SMAPE: 33.36
--------------------------------------------------

Testing LSTM on dataset Detached_House_2...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_2 using LSTM:
MAE: 377.99, RMSE: 545.58, SMAPE: 32.13
---------------------------

All done!


In [12]:
import os
import plotly.graph_objects as go
import plotly.express as px  # Import plotly express for color sequences

# Combined testing function
def combined_testing_and_visualization(datasets, models_list, loss_fn):
    metrics_data = {
        'MAE': {},
        'RMSE': {},
        'SMAPE': {}
    }

    for model_name, model in models_list.items():
        for dataset_name in datasets['test'].keys():
            print(f"\nTesting {model_name} on dataset {dataset_name}...\n")
            
            # Load the trained model's state_dict
            model.load_state_dict(torch.load(f'./Combine_MSE/{model_name}_{loss_fn}_{dataset_name}.pth'))
            model.to(device)

            test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)
            test_loss, test_metrics, actual_values, predicted_values = combined_test(model, test_loader, loss_fn)

            # Print test metrics
            print(f"Test Metrics for {dataset_name} using {model_name}:")
            print(f"MAE: {test_metrics['MAE']:.2f}, RMSE: {test_metrics['RMSE']:.2f}, SMAPE: {test_metrics['SMAPE']:.2f}")
            print('-' * 50)

            # Store metrics data for bar chart
            for metric in ['MAE', 'RMSE', 'SMAPE']:
                if dataset_name not in metrics_data[metric]:
                    metrics_data[metric][dataset_name] = []
                metrics_data[metric][dataset_name].append(test_metrics[metric])

    # Create bar chart for each metric and dataset
    for metric in ['MAE', 'RMSE', 'SMAPE']:
        for dataset_name in datasets['test'].keys():
            fig = go.Figure()

            sorted_metrics_data = sorted(list(zip(list(models_list.keys()), metrics_data[metric][dataset_name])),
                                         key=lambda x: x[1])  # Sort the data by metric values
            model_names_sorted, metrics_values_sorted = zip(*sorted_metrics_data)  # Unpack sorted data

            #colors = ['#00FE35', '#00FC55', '#00FA75', '#FF9090', '#FF6060', '#FF3030']  # Colors for bars
            colors = ['#00FE35', '#28F771', '#FF7975']  # Updated colors for bars

            fig.add_trace(go.Bar(
                y=model_names_sorted,  # Changed from x to y for horizontal bar
                x=metrics_values_sorted,  # Changed from y to x for horizontal bar
                orientation='h',  # Added for horizontal bar
                marker_color=colors  # Assigning colors to bars
            ))

            # Update layout
            fig.update_layout(
                title={
                    'text': f'Comparison of {metric} for {dataset_name}',
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                yaxis_title='Models',
                xaxis_title=metric,
                yaxis=dict(
                    tickvals=list(range(len(models_list.keys()))), 
                    ticktext=model_names_sorted,
                    title_standoff=15,
                    title_font=dict(size=20, color='black'),  # Updated size and color
                    tickfont=dict(size=16, color='black')    # Updated size and color
                ),
                xaxis=dict(
                    title_standoff=15,
                    title_font=dict(size=20, color='black'),  # Updated size and color
                    tickfont=dict(size=16, color='black')    # Updated size and color
                ),
                bargap=0.15,
                height=600
            )

            
            fig.show()
        # Create line plots for comparison with HyperForecasting model
        hyper_model = 'Hyperforecasting'
        for dataset_name in datasets['test'].keys():
            hyper_data = pd.read_csv(f"./Combine_MSE/{dataset_name}_{hyper_model}_results.csv")['Predicted'].values[:500]
            actual_data = pd.read_csv(f"./Combine_MSE/{dataset_name}_{hyper_model}_results.csv")['Actual'].values[:500]
    
            for model_name in models_list.keys():
                if model_name != hyper_model:  # Exclude HyperForecasting model from the comparison
                    comparison_data = pd.read_csv(f"./Combine_MSE/{dataset_name}_{model_name}_results.csv")['Predicted'].values[:500]
    
                    # Create a line plot
                    fig = go.Figure()
    
                    # Add traces for HyperForecasting, actual values, and the comparison model
                    fig.add_trace(go.Scatter(y=hyper_data, mode='lines', name='HyperForecasting', line=dict(color='#00FE35')))
                    fig.add_trace(go.Scatter(y=actual_data, mode='lines', name='Actual Values', line=dict(color='blue')))
                    fig.add_trace(go.Scatter(y=comparison_data, mode='lines', name=model_name, line=dict(color='red')))
    
                    # Update layout
                    fig.update_layout(
                        title=f"Comparison of Predictions for {dataset_name} - Hyper vs {model_name}",
                        xaxis_title="Time Steps",
                        yaxis_title="Values"
                    )
    
                    # Show the plot
                    fig.show()

        for dataset_name in datasets['test'].keys():
            fig = go.Figure()
    
            for model_name in models_list.keys():
                prediction_data = pd.read_csv(f"./Combine_MSE/{dataset_name}_{model_name}_results.csv")['Predicted'].values[:500]
                
                fig.add_trace(go.Box(y=prediction_data, name=model_name))
            
            # Update layout
            fig.update_layout(
                title={
                    'text': f'Distribution of Predictions for {dataset_name}',
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                yaxis_title="Predicted Values",
                xaxis_title="Models"
            )
    
            # Show the plot
            fig.show()

    print("All done!")

# Running the combined testing and visualization function
combined_testing_and_visualization(datasets, models_list, loss_fn)


Testing GRU on dataset Detached_House_1...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_1 using GRU:
MAE: 645.24, RMSE: 717.64, SMAPE: 32.60
--------------------------------------------------

Testing GRU on dataset Detached_House_2...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_2 using GRU:
MAE: 371.69, RMSE: 537.68, SMAPE: 31.47
--------------------------------------------------

Testing LSTM on dataset Detached_House_1...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_1 using LSTM:
MAE: 660.32, RMSE: 748.26, SMAPE: 33.36
--------------------------------------------------

Testing LSTM on dataset Detached_House_2...

Output shape: torch.Size([1, 24, 1])
Target shape: torch.Size([1, 24, 1])
Test Metrics for Detached_House_2 using LSTM:
MAE: 377.99, RMSE: 545.58, SMAPE: 32.13
---------------------------

All done!
